# Reward-driven learning of recurrent dynamics

In [ ]:
#!pip install ANNarchy



Reward-modulated recurrent network based on:

> Miconi T. (2017). Biologically plausible learning in recurrent neural networks reproduces neural dynamics observed during cognitive tasks. eLife 6:e20899. doi:10.7554/eLife.20899

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import ANNarchy as ann
ann.clear()
ann.setup(dt=1.0)

Each neuron in the reservoir follows the following equations:

$$
    \tau \frac{dx(t)}{dt} + x(t) = \sum_\text{input} W^\text{IN} \, r^\text{IN}(t) + \sum_\text{rec} W^\text{REC} \, r(t) + \xi(t)
$$

$$
    r(t) = \tanh(x(t))
$$

where $\xi(t)$ is a random perturbation at 3 Hz, with an amplitude randomly sampled between $-A$ and $+A$.

We additionally keep track of the mean firing rate with a sliding average:

$$
    \tilde{x}(t) = \alpha \, \tilde{x}(t) + (1 - \alpha) \, x(t)
$$

The three first neurons keep a constant rate throughout learning (1 or -1) to provide some bias to the other neurons.

In [ ]:
neuron = ann.Neuron(
    parameters = """
        tau = 30.0 : population # Time constant
        constant = 0.0 # The four first neurons have constant rates
        alpha = 0.05 : population # To compute the sliding mean
        f = 3.0 : population # Frequency of the perturbation
        A = 16. : population # Perturbation amplitude. dt*A/tau should be 0.5...
    """,
    equations="""
        # Perturbation
        perturbation = if Uniform(0.0, 1.0) < f/1000.: 1.0 else: 0.0 
        noise = if perturbation > 0.5: A * Uniform(-1.0, 1.0) else: 0.0

        # ODE for x
        x += dt*(sum(in) + sum(exc) - x + noise)/tau

        # Output r
        rprev = r # store r at previous time step
        r = if constant == 0.0: tanh(x) else: tanh(constant)

        # Sliding mean
        delta_x = x - x_mean
        x_mean = alpha * x_mean + (1 - alpha) * x
    """
)

The learning rule is defined by a trace $e_{i, j}(t)$ for each synapse $i \rightarrow j$ incremented at each time step with:

$$
    e_{i, j}(t) = e_{i, j}(t-1) + (r_i (t) \, x_j(t))^3
$$

At the end $T$ of a trial, the reward $R$ is delivered and all weights are updated using:

$$
    \Delta w_{i, j} = \eta \,  e_{i, j}(T) \, |R_\text{mean}| \,  (R - R_\text{mean})
$$

where $R_\text{mean}$ is the mean reward for the task. Here the reward is defined as the opposite of the prediction error.

All traces are then reset to 0 for the next trial. Weight changes are clamped between -0.0003 and 0.0003.

As ANNarchy applies the synaptic equations at each time step, we need to introduce a boolean `learning_phase` which performs trace integration when 0, weight update when 1.

In [ ]:
synapse = ann.Synapse(
    parameters="""
        eta = 0.5 : projection # Learning rate
        learning_phase = 0.0 : projection # Flag to allow learning only at the end of a trial
        reward = 0.0 : projection # Reward received
        mean_reward = 0.0 : projection # Mean Reward received
        max_weight_change = 0.0003 : projection # Clip the weight changes
    """,
    equations="""
        # Trace
        trace += if learning_phase < 0.5:
                    power(pre.rprev * (post.delta_x), 3)
                 else:
                    0.0

        # Weight update only at the end of the trial
        delta_w = if learning_phase > 0.5:
                eta * trace * fabs(mean_reward) * (reward - mean_reward)
             else:
                 0.0 : min=-max_weight_change, max=max_weight_change
        w += delta_w
        
    """
)

The RC network has two inputs A and B. The reservoir has 200 neurons, 3 of which having constant rates to serve as biases for the other neurons. 

In [ ]:
# Input population
inp = ann.Population(2, ann.Neuron(parameters="r=0.0"))

# Recurrent population
N = 200
pop = ann.Population(N, neuron)

# Biases
pop[0].constant = 1.0
pop[1].constant = 1.0
pop[2].constant = -1.0

Input weights are uniformly distributed between -1 and 1.

The recurrent weights are normally distributed, with a coupling strength of $g=1.5$ (edge of chaos). In the original paper, the projection is fully connected (but self-connections are avoided). Using a sparse (0.1) connectivity matrix leads to similar results and is much faster.

In [ ]:
# Input weights
Wi = ann.Projection(inp, pop, 'in')
Wi.connect_all_to_all(weights=ann.Uniform(-1.0, 1.0))

# Recurrent weights
g = 1.5
sparseness = 0.1

Wrec = ann.Projection(pop, pop, 'exc', synapse)
if sparseness == 1.0:
    Wrec.connect_all_to_all(weights=ann.Normal(0., g/np.sqrt(N)))
else:
    Wrec.connect_fixed_probability(probability=sparseness, weights=ann.Normal(0., g/np.sqrt(sparseness*N)))

In [ ]:
ann.compile()

The output of the reservoir is chosen to be the neuron of index 100.

In [ ]:
OUTPUT_NEURON = 100

We record the rates inside the reservoir:

In [ ]:
m = ann.Monitor(pop, ['r'], start=False)

We model the DNMS (delayed non-match-to-sample) task used by Miconi in the paper. A stimulus (`A` or `B`) is presented for `d_stim = 200` ms followed by a delay period `d_delay = 200` ms. A second stimulus (`A` or `B`) is presented again. After another delay period, the activity of the output neuron is recorded for `d_response = 200`ms. If the two stimulus were identical, the mean activity of the output neuron during the response period should be close to -1, otherwise it should be close to +1. 

Parameters defining the duration of the task:

In [ ]:
# Durations
d_stim = 200
d_delay= 200
d_response = 200

Definition of a DNMS trial (AA, AB, BA, BB):

In [ ]:
def dnms_trial(
        input:list, # List of integers (0 or 1) for the inputs: AA, AB, BA, BB
        target:float, # Target activity for the output neuron (+- 0.98)
        R_mean:float, # Mean reward 
        learning:bool = True, # Learning enabled
        record:bool = False, # Recording enabled
        perturbation:bool = True # Perturbations enabled
    ):

    # Switch off perturbations if needed
    if not perturbation:
        old_A = pop.A
        pop.A = 0.0

    # Reinitialize network
    pop.x = ann.Uniform(-0.1, 0.1).get_values(N)
    pop.r = np.tanh(pop.x)
    pop[0].r = np.tanh(1.0)
    pop[1].r = np.tanh(1.0)
    pop[2].r = np.tanh(-1.0)

    if record: m.resume()

    # First input
    inp[input[0]].r = 1.0
    ann.simulate(d_stim)
    
    # Delay
    inp.r = 0.0
    ann.simulate(d_delay)
    
    # Second input
    inp[input[1]].r = 1.0
    ann.simulate(d_stim)
    
    # Delay
    inp.r = 0.0
    ann.simulate(d_delay)
    
    # Response
    if not record: m.resume()
    inp.r = 0.0
    ann.simulate(d_response)
    
    # Read the output
    m.pause()
    recordings = m.get('r')
    
    # Response is over the last 200 ms
    output = recordings[-int(d_response):, OUTPUT_NEURON] # neuron 100 over the last 200 ms
    
    # Compute the reward as the opposite of the absolute error
    reward = - np.mean(np.abs(target - output))
    
    # The first 25 trial do not learn, to let R_mean get realistic values
    if learning:

        # Apply the learning rule
        Wrec.learning_phase = 1.0
        Wrec.reward = reward
        Wrec.mean_reward = R_mean

        # Learn for one step
        ann.step()
        
        # Reset the traces
        Wrec.learning_phase = 0.0
        Wrec.trace = 0.0
        _ = m.get() # to flush the recording of the last step

    # Switch back on perturbations if needed
    if not perturbation:
        pop.A = old_A

    return recordings, reward

Let's visualize the activity of the output neuron during the first four trials without learning (AA, AB, BA and BB). 

In [ ]:
# Perform the four different trials successively
initialAA, errorAA = dnms_trial(input=[0, 0], target=-0.98, R_mean=0.0, learning=False, record=True, perturbation=True)
initialAB, errorAB = dnms_trial(input=[0, 1], target=+0.98, R_mean=0.0, learning=False, record=True, perturbation=True)
initialBA, errorBA = dnms_trial(input=[1, 0], target=+0.98, R_mean=0.0, learning=False, record=True, perturbation=True)
initialBB, errorBB = dnms_trial(input=[1, 1], target=-0.98, R_mean=0.0, learning=False, record=True, perturbation=True)

plt.figure(figsize=(12, 10))
ax = plt.subplot(221)
ax.plot(initialAA[:, OUTPUT_NEURON], lw=3)
ax.plot(initialAA[:, OUTPUT_NEURON + 1], lw=1)
ax.plot(initialAA[:, OUTPUT_NEURON + 2], lw=1)
ax.plot(initialAA[:, OUTPUT_NEURON + 3], lw=1)
ax.set_ylim((-1., 1.))
ax.set_title('Output AA -1')
ax = plt.subplot(222)
ax.plot(initialBA[:, OUTPUT_NEURON], lw=3)
ax.plot(initialBA[:, OUTPUT_NEURON + 1], lw=1)
ax.plot(initialBA[:, OUTPUT_NEURON + 2], lw=1)
ax.plot(initialBA[:, OUTPUT_NEURON + 3], lw=1)
ax.set_ylim((-1., 1.))
ax.set_title('Output BA +1')
ax = plt.subplot(223)
ax.plot(initialAB[:, OUTPUT_NEURON], lw=3)
ax.plot(initialAB[:, OUTPUT_NEURON + 1], lw=1)
ax.plot(initialAB[:, OUTPUT_NEURON + 2], lw=1)
ax.plot(initialAB[:, OUTPUT_NEURON + 3], lw=1)
ax.set_ylim((-1., 1.))
ax.set_title('Output AB +1')
ax = plt.subplot(224)
ax.plot(initialBB[:, OUTPUT_NEURON], lw=3)
ax.plot(initialBB[:, OUTPUT_NEURON + 1], lw=1)
ax.plot(initialBB[:, OUTPUT_NEURON + 2], lw=1)
ax.plot(initialBB[:, OUTPUT_NEURON + 3], lw=1)
ax.set_ylim((-1., 1.))
ax.set_title('Output BB -1')

plt.show()

Depending on the initial weights, the activity of the output neuron during the last 200ms is different from the desired values. 

**Q:** Perform the simulation for 5000 groups of learning trials (AA, AB. BA and BB). Beware, this can take 15 to 20 minutes.

You will need to update the mean reward `R_mean` for each of the four trials separately (AA, AB, BA, BB). The mean reward can be computed as a simple sliding average of the reward:

$$R_\text{mean}(\text{AA}) = \alpha \, R_\text{mean}(\text{AA}) + (1 - \alpha) \ r$$
$$R_\text{mean}(\text{AB}) = \alpha \, R_\text{mean}(\text{AB}) + (1 - \alpha) \ r$$
$$R_\text{mean}(\text{BA}) = \alpha \, R_\text{mean}(\text{BA}) + (1 - \alpha) \ r$$
$$R_\text{mean}(\text{BB}) = \alpha \, R_\text{mean}(\text{BB}) + (1 - \alpha) \ r$$

where $r$ is the reward received at the end of each trial and $\alpha=0.75$. The mean reward can be initialized to -1. 

The first 25 trials should be made without learning, so that `R_mean`has time to reach a meaningful value. 

Record the rewards received during each trial and make a plot. Is learning fast or slow? Stable or unstable? Compare with FORCE learning.

**Q:** Visualize the activity of the output neuron during the four trials after training, without learning nor perturbation.